In [1]:
from PIL import Image
import numpy as np
from matplotlib import pyplot as plt

In [2]:
room_categories_path = "/home/cgokmen/room_categories.txt"
with open(room_categories_path) as f:
    sem_to_id = {line.strip(): i + 1 for i, line in enumerate(f.readlines())}
id_to_sem = {v: k for k, v in sem_to_id.items()}

In [3]:
# Check that all room types that show up on the list also show up on the image
import json

with open("/scr/ig_pipeline/artifacts/pipeline/combined_room_object_list.json") as f:
    room_object_lists = json.load(f)["scenes"]
scene_room_instances = {scene: list(rooms.keys()) for scene, rooms in room_object_lists.items()}
scene_room_types = {scene: {x.rsplit("_", 1)[0] for x in rooms} for scene, rooms in scene_room_instances.items()}
assert all(room in sem_to_id for rooms in scene_room_types.values() for room in rooms)

In [4]:
scene_room_types

{'Beechwood_0_int': {'bathroom',
  'corridor',
  'dining_room',
  'kitchen',
  'living_room',
  'private_office',
  'storage_room',
  'utility_room'},
 'Beechwood_1_int': {'bathroom',
  'bedroom',
  'childs_room',
  'closet',
  'corridor',
  'playroom',
  'storage_room',
  'television_room'},
 'Benevolence_0_int': {'bathroom', 'corridor', 'empty_room', 'entryway'},
 'Benevolence_1_int': {'corridor',
  'dining_room',
  'kitchen',
  'living_room',
  'pantry_room',
  'storage_room'},
 'Benevolence_2_int': {'bathroom', 'bedroom', 'closet', 'corridor'},
 'Ihlen_0_int': {'bathroom',
  'corridor',
  'dining_room',
  'garage',
  'living_room',
  'storage_room'},
 'Ihlen_1_int': {'bathroom',
  'bedroom',
  'corridor',
  'dining_room',
  'kitchen',
  'living_room',
  'staircase',
  'storage_room'},
 'Merom_0_int': {'bathroom',
  'childs_room',
  'closet',
  'living_room',
  'playroom',
  'storage_room',
  'utility_room'},
 'Merom_1_int': {'bathroom',
  'bedroom',
  'childs_room',
  'closet',
  '

In [5]:
import fs.zipfs
import fs.path
maps_fs = fs.zipfs.ZipFS("/scr/ig_pipeline/artifacts/parallels/maps.zip")
valid_scenes = set(maps_fs.listdir("/scenes"))
scene_maps = {
    scene: np.array(Image.open(maps_fs.open(fs.path.join("scenes", scene, "layout", "floor_semseg_0.png"), "rb"))).astype(int)
    for scene in valid_scenes
}

In [6]:
valid_scenes

{'Beechwood_0_int',
 'Beechwood_1_int',
 'Benevolence_0_int',
 'Benevolence_1_int',
 'Benevolence_2_int',
 'Ihlen_0_int',
 'Ihlen_1_int',
 'Merom_0_int',
 'Merom_1_int',
 'Pomaria_0_garden',
 'Pomaria_0_int',
 'Pomaria_1_int',
 'Pomaria_2_int',
 'Rs_int',
 'Wainscott_0_int',
 'Wainscott_1_int',
 'grocery_store_cafe',
 'house_double_floor_lower',
 'house_double_floor_upper',
 'house_single_floor',
 'office_large',
 'restaurant_brunch'}

In [7]:
room_ids_in_map = {scene: set(np.unique(img)) - {0} for scene, img in scene_maps.items()}
room_types_in_map = {scene: {id_to_sem[x] for x in room_ids} for scene, room_ids in room_ids_in_map.items()}

In [8]:
for scene in valid_scenes:
    expected = scene_room_types[scene]
    found = room_types_in_map[scene]
    if expected != found:
        print("Problem in", scene)
        print("Expected", expected)
        print("Found", found)
        print("Missing", expected - found)
        print("Unexpected", found - expected)
        print()

Problem in house_single_floor
Expected {'garage', 'garden', 'kitchen', 'childs_room', 'empty_room', 'living_room', 'sauna', 'entryway', 'dining_room', 'corridor', 'closet', 'bedroom', 'utility_room', 'bathroom'}
Found {'garage', 'garden', 'kitchen', 'childs_room', 'living_room', 'sauna', 'closet', 'bedroom', 'utility_room', 'bathroom'}
Missing {'entryway', 'empty_room', 'dining_room', 'corridor'}
Unexpected set()

Problem in grocery_store_cafe
Expected {'grocery_store', 'dining_room', 'bar'}
Found {'grocery_store', 'dining_room'}
Missing {'bar'}
Unexpected set()

Problem in Benevolence_0_int
Expected {'entryway', 'corridor', 'empty_room', 'bathroom'}
Found {'corridor', 'empty_room', 'bathroom'}
Missing {'entryway'}
Unexpected set()

Problem in house_double_floor_upper
Expected {'television_room', 'bedroom', 'bathroom'}
Found {'television_room', 'bedroom'}
Missing {'bathroom'}
Unexpected set()

Problem in Wainscott_1_int
Expected {'playroom', 'exercise_room', 'storage_room', 'private_of